## Interaktion mehrerer Agenten

In diesem Ipython Notebook werden wir uns ansehen, wie meherere Agenten miteinander kommunizieren können. Dadurch lassen sich Aufgaben aufteilen oder Aufgabestellungen aus verschiedenen Perspektiven betrachten.
Wir setzen in diesem Beispiel das Framework CrewAI ein. Mit Crew AI lassen sich einfach Teams aus mehreren Agenten zusammensetzten, um diese gemeinsam an Problemstellungen arbeiten zu lasen.


In [ ]:
%pip install -r requirements.txt

In [ ]:
from dotenv import load_dotenv

load_dotenv()

Zuerst Stellen wir uns ein Team von Agenten zusammen. Hierfür überlegen wir uns zuerst Namen. Zusätzlich teilen wir den Agenten Tools zu, die sie einsetzen dürfen.


In [ ]:
names = {
    "AI visionary": ["tavily_search"],
    "Grumpy old senior developer": ["arxiv", "tavily_search"],
    "Junior Software developer": ["tavily_search"],
}

Wir überlegen uns ein Thema, über das die Agenten diskutieren sollen.
Mittels LLM generieren wir eine ausgearbeitete Variante des Diskussionsthemas.


In [ ]:
from langchain_core.messages import (
    HumanMessage,
    SystemMessage,
    BaseMessage,
)

from helpers import llm

topic = "The current impact of automation and artificial intelligence on the employment situation of software developers"
word_limit = 50

topic_specifier_prompt = [
    SystemMessage(content="You can make a topic more specific."),
    HumanMessage(
        content=f"""{topic}
        
        You are the moderator.
        Please make the topic more specific.
        Please reply with the specified quest in {word_limit} words or less. 
        Speak directly to the participants: {*names,}.
        Do not add anything else."""
    ),
]
specified_topic = llm()(topic_specifier_prompt).content

Anhand der Namen der Agenten und des Diskussionsthemas lassen wir uns per LLM für jeden Agenten eine ausführliche Rollenbeschreibung generieren.


In [ ]:
conversation_description = f"""Here is the topic of conversation: {topic}
The participants are: {', '.join(names.keys())}"""

agent_descriptor_system_message = SystemMessage(
    content="You can add detail to the description of the conversation participant."
)


def generate_agent_description(name):
    agent_specifier_prompt = [
        agent_descriptor_system_message,
        HumanMessage(
            content=f"""{conversation_description}
            Please reply with a creative description of {name}, in {word_limit} words or less. 
            Speak directly to {name}.
            Give them a point of view.
            Do not add anything else."""
        ),
    ]
    agent_description = llm(openai_model="gpt-3.5-turbo")(
        agent_specifier_prompt
    ).content

    return agent_description


agent_descriptions = {name: generate_agent_description(name) for name in names}


def generate_system_message(name, description, tools):
    return f"""{conversation_description}
    
Your name is {name}.

Your description is as follows: {description}

Your goal is to persuade your conversation partner of your point of view.

DO look up information with your tool to refute your partner's claims.
You can use the following tools: {', '.join(tools)}.
DO cite your sources.

DO NOT fabricate fake citations.
DO NOT cite any source that you did not look up.

Do not add anything else.

Stop speaking the moment you finish speaking from your perspective.
"""


agent_system_messages = {
    name: generate_system_message(name, description, tools)
    for (name, tools), description in zip(names.items(), agent_descriptions.values())
}
for agent_system_message in agent_system_messages:
    print(agent_system_message)

Die eingesetzten Tools müssen importiert werden, damit sie von den Agenten eingesetzt werden können.


In [ ]:
from langchain_community.tools.arxiv.tool import ArxivQueryRun
from langchain_community.tools.tavily_search.tool import TavilySearchResults

available_tools = {
    "tavily_search": TavilySearchResults(max_results=1),
    "arxiv": ArxivQueryRun(),
}

Für jedes Crew Member erstellen wir in diesem Schritt einen Langchain Agenten.


In [ ]:
from crewai import Agent


class DiscussionAgents:
    def __init__(self, names):
        self.names = names

    def speaker_agents(self):
        agents = {}
        for name, agent_tools in self.names.items():
            agents[name] = Agent(
                role=f"{name}",
                goal=agent_system_messages[name],
                backstory="You always respond directly to the actual discussion in your own way.",
                verbose=False,
                allow_delegation=False,
                tools=[
                    available_tools[name]
                    for name in agent_tools
                    if name in available_tools
                ],
            )
        return agents


discussion_agents = DiscussionAgents(names)
agents = discussion_agents.speaker_agents()

Außerdem benötigt jeder Teilnehmer einen Task der grob beschreibt, welche Aufgabe das Crew Mitglied hat.


In [ ]:
from crewai import Task


class DiscussionTasks:
    def speaker_task(self, agent):
        return Task(
            description=f"You are {agent.role}. You participate in a discussion. Always directly respond to the opinions of the other speakers. Always call other speakers by name, when you respond to them.",
            agent=agent,
            expected_output="Output your opinion in 40 words or less. Do not output Sources.",
        )

Jetzt wird die Crew erstellt. Zur Crew werden die einzelnen Member und ihre Tasks hinzugefügt. Zusätzlich können weitere Parameter zum Verhalten der Crew konfiguriert werden.


In [ ]:
from crewai import Crew, Process
from langchain_core.messages import ChatMessage


class DiscussionCrew:
    def __init__(self):
        agents = DiscussionAgents(names)
        self.speaker_agents = []
        for name in agents.speaker_agents():
            attr_name = name.replace(" ", "_").replace(
                ".", "").replace(",", "")
            agent = agents.speaker_agents()[name]
            setattr(self, f"speaker_agent_{attr_name}", agent)
            self.speaker_agents.append(agent)

    def print_final_answer(_, intermediate_steps):
        if hasattr(intermediate_steps, "log"):
            log = intermediate_steps.log
            final_answer_index = log.find("Final Answer:")
            final_answer = log[final_answer_index +
                               len("Final Answer:"):].strip()
            print(final_answer)
        else:
            return

    def kickoff(self, state):
        print("The discussion is about to start.")
        print("-------------------------------")
        tasks = DiscussionTasks()
        crew = Crew(
            agents=self.speaker_agents,
            tasks=[tasks.speaker_task(agent) for agent in self.speaker_agents],
            verbose=1,
            full_output=True,
            process=Process.sequential,
            step_callback=self.print_final_answer,
        )
        result = crew.kickoff()
        output_messages = []
        for output in result["tasks_outputs"]:
            description = output.description
            role = description.replace("You are ", "", 1)
            role = role.split(".", 1)[0]
            output_messages.append(
                ChatMessage(content=output.exported_output, role=role)
            )

        return {"messages": output_messages}

Nachdem die Crew jetzt steht, müssen wir noch um den LangGraph Part kümmern. Der Graph sorgt dafür, dass die Crew über mehrere Runden diskutiert.

Hierfür definieren wir zuerst die Nodes des Graphen.


In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults


class Nodes:
    def __init__(self):
        self.tavily_search_tool = TavilySearchResults(max_results=3)

    def call_host(self, state):
        print("# Calling next speaker round")
        print("-------------------------------")
        turns = state.get("turns") or 0
        turns += 1

        return {"turns": turns}

    # Define the function that determines whether to continue or not
    def should_continue(self, state):
        turns = state["turns"]
        # TODO: change: if max rounds not reached, continue
        if turns <= 2:
            print("-- CONTINUE ---")
            return "continue"
        else:
            print("-- END ---")
            return "end"

Der Graph benötigt einen State, der über die einzelnen Nodes weitergereicht wird.


In [ ]:
import operator
from typing import Annotated, Sequence, TypedDict


class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]
    turns: int

Im Workflow wird der Graph zusammengesetzt und die Edges definiert.


In [ ]:
from langgraph.graph import StateGraph, END


class WorkFlow:
    def __init__(self):
        nodes = Nodes()
        workflow = StateGraph(AgentState)

        workflow.add_node("call_host", nodes.call_host)
        workflow.add_node("call_crew", DiscussionCrew().kickoff)

        workflow.set_entry_point("call_host")
        workflow.add_conditional_edges(
            "call_host", nodes.should_continue, {
                "continue": "call_crew", "end": END}
        )
        workflow.add_edge("call_crew", "call_host")
        self.app = workflow.compile()

Der Graph kann nun ausgeführt werden und die Crew beginnt zu diskutieren.


In [ ]:
app = WorkFlow().app
app.invoke({"messages": [ChatMessage(content=specified_topic, role="host")]})